## Prepared for the textbook:
-------------------------------------------------------------------
## Data Analysis for Business, Economics, and Policy
#### by Gabor BEKES and  Gabor KEZDI 
----------------------------------
#### Cambridge University Press 2021
-----------------------------------------------------------------------------------------------
#### License: Free to share, modify and use for educational purposes. Not to be used for business purposes.

In [1]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from plotnine import *
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from patsylearn import PatsyModel, PatsyTransformer

In [2]:
#pip install git+https://github.com/amueller/patsylearn.git

  Cloning https://github.com/amueller/patsylearn.git to /tmp/pip-req-build-4cztn6we
  Running command git clone -q https://github.com/amueller/patsylearn.git /tmp/pip-req-build-4cztn6we
  Created wheel for patsylearn: filename=patsylearn-0.0.1-py3-none-any.whl size=10710 sha256=52a553af885495b15bceb18628612c0430d14fdb0165222295cf7f9fc45e809e
  Stored in directory: /tmp/pip-ephem-wheel-cache-odarda7z/wheels/ee/04/89/4c256a72966ee1ab187dde02653b89e17b640d0e4240deaf81
Successfully built patsylearn
Note: you may need to restart the kernel to use updated packages.


In [3]:
path = Path(os.getcwd())

In [4]:
base_dir = path.parent.parent

In [5]:
data_in = os.path.join(str(base_dir) , "da_case_studies/ch16-airbnb-random-forest/")
data_out = os.path.join(str(base_dir), "da_case_studies/ch16-airbnb-random-forest/")
output = os.path.join(str(base_dir), "da_case_studies/ch16-airbnb-random-forest/output/")
func = os.path.join(str(base_dir) ,   "da_case_studies/ch00-tech-prep/")

In [6]:
sys.path.append(func)
from py_helper_functions import *

In [7]:
#########################################################################################
#
# PART I
# Loading and preparing data ----------------------------------------------
#
#########################################################################################

In [8]:
area="london"
data=pd.read_csv(data_in+"airbnb_"+area+"_workfile_adj.csv")
data=data.loc[data.price.notna(),:]

In [9]:
def count_missing_values(df):
    return df.isna().sum()[df.isna().sum()>0]

In [10]:
count_missing_values(data)

Series([], dtype: int64)

In [11]:
# Sample definition and preparation ---------------------------------------

# We focus on normal apartments, n<8
data = data.query('n_accommodates < 8')


In [12]:
# copy a variable - purpose later, see at variable importance
data = data.assign(n_accommodates_copy = data.n_accommodates)

In [13]:

# basic descr stat -------------------------------------------
data.describe()

,usd_price_day,n_accommodates,n_bathrooms,n_review_scores_rating,n_number_of_reviews,n_guests_included,n_reviews_per_month,n_extra_people,n_minimum_nights,n_beds,...,flag_review_scores_rating,flag_reviews_per_month,flag_n_number_of_reviews,ln_days_since,ln_days_since2,ln_days_since3,n_days_since2,n_days_since3,ln_review_scores_rating,n_accommodates_copy
count,49826.000000,49826.000000,49826.000000,49826.000000,49826.000000,49826.000000,49826.000000,49826.00000,49826.000000,49826.000000,...,49826.000000,49826.000000,49826.0,49826.000000,49826.000000,49826.000000,4.982600e+04,4.982600e+04,49826.000000,49826.000000
mean,88.984044,2.838759,1.224260,92.458616,12.340144,1.356180,1.138253,6.50873,3.300947,1.585959,...,0.318850,0.303998,0.0,5.713297,33.456371,199.607207,2.931985e+05,3.046539e+08,4.520910,2.838759
std,70.515089,1.494748,0.466448,8.418474,25.889935,0.879263,1.238156,12.31549,29.578062,0.927501,...,0.466035,0.459986,0.0,0.902564,9.506988,80.264045,5.640361e+05,9.636140e+08,0.120421,1.494748
min,8.000000,1.000000,0.000000,20.000000,0.000000,1.000000,0.010000,0.00000,1.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2.995732,1.000000
25%,41.000000,2.000000,1.000000,92.000000,0.000000,1.000000,0.470000,0.00000,1.000000,1.000000,...,0.000000,0.000000,0.0,5.433722,29.525335,160.432461,5.198400e+04,1.185235e+07,4.521789,2.000000
50%,70.000000,2.000000,1.000000,94.000000,3.000000,1.000000,0.770000,0.00000,2.000000,1.000000,...,0.000000,0.000000,0.0,5.793014,33.559007,194.407782,1.069290e+05,3.496578e+07,4.543295,2.000000
75%,110.000000,4.000000,1.000000,97.000000,12.000000,1.000000,1.170000,10.00000,3.000000,2.000000,...,1.000000,1.000000,0.0,6.224558,38.745128,241.171311,2.540160e+05,1.280241e+08,4.574711,4.000000
max,999.000000,7.000000,8.000000,100.000000,396.000000,16.000000,15.000000,240.00000,5000.000000,16.000000,...,1.000000,1.000000,0.0,7.909489,62.560024,494.817853,7.409284e+06,2.016807e+10,4.605170,7.000000


In [14]:
data

,f_property_type,f_room_type,f_cancellation_policy,f_bed_type,f_neighbourhood_cleansed,usd_price_day,n_accommodates,n_bathrooms,n_review_scores_rating,n_number_of_reviews,...,flag_review_scores_rating,flag_reviews_per_month,flag_n_number_of_reviews,ln_days_since,ln_days_since2,ln_days_since3,n_days_since2,n_days_since3,ln_review_scores_rating,n_accommodates_copy
0,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,23.0,1,1.0,100.0,1,...,0,0,0,4.532599,20.544458,93.119801,8464.0,778688.0,4.605170,1
1,Apartment,Private room,moderate,Couch,Kingston upon Thames,50.0,2,1.0,91.0,15,...,0,0,0,6.510258,42.383464,275.927298,450241.0,302111711.0,4.510860,2
2,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,24.0,2,1.0,80.0,2,...,0,0,0,5.493061,30.173724,165.746120,58564.0,14172488.0,4.382027,2
3,House,Private room,flexible,Real Bed,Kingston upon Thames,50.0,2,1.5,94.0,0,...,1,1,0,5.793014,33.559007,194.407782,106929.0,34965783.0,4.543295,2
4,House,Private room,flexible,Real Bed,Kingston upon Thames,25.0,1,1.0,94.0,0,...,1,1,0,5.793014,33.559007,194.407782,106929.0,34965783.0,4.543295,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51641,Apartment,Private room,strict,Real Bed,Westminster,24.0,1,1.0,40.0,1,...,0,0,0,5.181784,26.850881,139.135452,31329.0,5545233.0,3.688879,1
51642,House,Entire home/apt,flexible,Real Bed,Richmond upon Thames,120.0,6,1.0,94.0,0,...,1,1,0,5.793014,33.559007,194.407782,106929.0,34965783.0,4.543295,6
51643,Apartment,Entire home/apt,flexible,Real Bed,Wandsworth,215.0,6,3.0,94.0,0,...,1,1,0,5.793014,33.559007,194.407782,106929.0,34965783.0,4.543295,6
51644,Apartment,Entire home/apt,flexible,Real Bed,Hounslow,150.0,4,2.0,94.0,0,...,1,1,0,5.793014,33.559007,194.407782,106929.0,34965783.0,4.543295,4


In [15]:
data.price.describe()

count    49826.000000
mean        88.984044
std         70.515089
min          8.000000
25%         41.000000
50%         70.000000
75%        110.000000
max        999.000000
Name: price, dtype: float64

In [16]:
data.f_room_type.value_counts()

Entire home/apt    24938
Private room       24351
Shared room          537
Name: f_room_type, dtype: int64

In [17]:
data.f_property_type.value_counts()

Apartment    37467
House        12359
Name: f_property_type, dtype: int64

In [18]:
data.f_number_of_reviews.value_counts()

1    31576
0    15147
2     3103
Name: f_number_of_reviews, dtype: int64

In [19]:
# create train and holdout samples -------------------------------------------
# train is where we do it all, incl CV

# First pick a smaller than usual training set so that models run faster and check if works
# If works, start anew without these two lines

In [20]:
data_train, data_holdout = train_test_split(data,train_size=0.7, random_state=2801)

In [21]:
data_train.shape, data_holdout.shape

((34878, 91), (14948, 91))

In [22]:
# Basic Variables inc neighnourhood
basic_vars = [
  "n_accommodates", "n_beds", "n_days_since",
  "f_property_type","f_room_type", "f_bathroom", "f_cancellation_policy", "f_bed_type",
  "f_neighbourhood_cleansed"]

# reviews
reviews = ["n_number_of_reviews", "flag_n_number_of_reviews" ,"n_review_scores_rating", "flag_review_scores_rating"]

# Dummy variables
amenities =  [col for col in data if col.startswith('d_')]

#interactions for the LASSO
# from ch14
X1  = ["n_accommodates:f_property_type",  "f_room_type:f_property_type",  "f_room_type:d_familykidfriendly",
         "d_airconditioning:f_property_type", "d_cats:f_property_type", "d_dogs:f_property_type"]
# with boroughs
X2  = ["f_property_type:f_neighbourhood_cleansed", "f_room_type:f_neighbourhood_cleansed",
         "n_accommodates:f_neighbourhood_cleansed" ]

In [23]:
predictors_1 = basic_vars
predictors_2 = basic_vars+reviews+amenities
predictors_E = basic_vars+reviews+amenities+X1+X2

In [24]:
## Testing patsylearn data transform
PatsyTransformer("n_accommodates:f_property_type", return_type="dataframe").fit_transform(data).head(2)

,n_accommodates:f_property_type[Apartment],n_accommodates:f_property_type[House]
0,1.0,0.0
1,2.0,0.0


In [25]:
#########################################################################################
#
# PART II
# RANDOM FORESTS -------------------------------------------------------
#
#########################################################################################

In [26]:
from skranger.ensemble import RangerForestRegressor

In [27]:
rfr = RangerForestRegressor()

In [34]:
rf_model_1=PatsyModel(rfr, "price ~ " + " + ".join(predictors_1))
rf_model_2=PatsyModel(rfr, "price ~ " + " + ".join(predictors_2))

In [35]:
rf_model_1.fit(data_train)
rf_model_2.fit(data_train)

PatsyModel(NA_action='drop', add_intercept=False,
           estimator=RangerForestRegressor(alpha=0.5,
                                           always_split_features=None,
                                           categorical_features=None,
                                           holdout=False, importance='none',
                                           inbag=None, keep_inbag=False,
                                           local_importance=False, max_depth=0,
                                           min_node_size=0, minprop=0.1, mtry=0,
                                           n_estimators=100, n_jobs=-1,
                                           num_random_splits=1, oob_error=False,
                                           regularization...
                   'd_lockonbedroomdoor + d_lockbox + d_otherpets + '
                   'd_paidparkingoffpremises + d_petsallowed + '
                   'd_petsliveonthisproperty + d_pool + d_privateentrance + '
                 

In [36]:
rf_model_1

PatsyModel(NA_action='drop', add_intercept=False,
           estimator=RangerForestRegressor(alpha=0.5,
                                           always_split_features=None,
                                           categorical_features=None,
                                           holdout=False, importance='none',
                                           inbag=None, keep_inbag=False,
                                           local_importance=False, max_depth=0,
                                           min_node_size=0, minprop=0.1, mtry=0,
                                           n_estimators=100, n_jobs=-1,
                                           num_random_splits=1, oob_error=False,
                                           regularization...
                                           respect_categorical_features=None,
                                           sample_fraction=None,
                                           save_memory=False,
                         

In [32]:
from sklearn.inspection import plot_partial_dependence

In [33]:
plot_partial_dependence(rf_model_1)

TypeError: plot_partial_dependence() missing 2 required positional arguments: 'X' and 'features'